# Census Data Tools

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

In [1]:
import morpc

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [2]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [3]:
api = morpc.census.api_get(url, params)

Total variables requested: 3
Starting request #1. 3 variables remain.


In [4]:
api

,NAME,B01001_001E
GEO_ID,,
0500000US39041,"Delaware County, Ohio",226296
0500000US39049,"Franklin County, Ohio",1321820


## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [5]:
acs = morpc.census.acs_data('B11001', '2023', '5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [7]:
acs.DIMENSIONS

['TOTAL', 'Household Type', 'Living Alone', 'Spouse Present']

In [6]:
acs.VARS

{'B11001_001E': {'label': 'Estimate!!Total:',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_001EA,B11001_001M,B11001_001MA'},
 'B11001_002E': {'label': 'Estimate!!Total:!!Family households:',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_002EA,B11001_002M,B11001_002MA'},
 'B11001_003E': {'label': 'Estimate!!Total:!!Family households:!!Married-couple family',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_003EA,B11001_003M,B11001_003MA'},
 'B11001_004E': {'label': 'Estimate!!Total:!!Family households:!!Other family:',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_004EA,B11001_004M,B11001_004MA'},
 'B11001_005E': {'

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 

#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [ ]:
morpc.census.SCOPES

In [ ]:
morpc.census.ACS_VAR_GROUPS['B11001']['dimensions']

In [ ]:
acs = acs.query(scope='region15-counties')

In [ ]:
data = acs.DATA
data.head()

### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

In [ ]:
acs = acs.query(get_param=['B11001_001E'], for_param='state:39')

### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [ ]:
acs = morpc.census.acs_data('B01001', '2023', '5')
acs = acs.query(scope='region15-counties', get_param=['B01001_002E', 'B01001_026E'])
acs.DIM_TABLE

### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [ ]:
acs.save(output_dir='./temp_data/')

In [ ]:
acs.SCHEMA

In [ ]:
acs.RESOURCE

## Georeference the data to map

In [ ]:
acs = morpc.census.acs_data('B01001', '2023', '5')
acs = acs.query(get_param=['B01001_001E'], scope='region15-tracts')

In [ ]:
acs = acs.georeference()

In [ ]:
acs.DATA.explore(column='B01001_001E')

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()

In [ ]:
acs.API_URL 

### Build ACS Variable Group JSON for Dimension names

In [8]:
import requests
r = requests.get('https://api.census.gov/data/2023/acs/acs5/variables.json')
varjson = r.json()

In [ ]:
groups = {}
for variable in varjson['variables']:
    if variable not in ['for', 'in', 'ucgid', 'GEO_ID', 'AIANHH', 'AIHHTL', 'AIRES', 'ANRC']:
        group = varjson['variables'][variable]['group']
        if not group[-1].isalpha():
            if group not in groups:
                groups[group] = {}
                groups[group]['concept'] = varjson['variables'][variable]['concept']
                groups[group]['dimensions'] = ['TOTAL'] + varjson['variables'][variable]['concept'].replace(' by ',':').replace(' and ',':').split(':')
                variables = {}
                for variable in varjson['variables']:
                    if varjson['variables'][variable]['group'] == group:
                        variables[variable] = varjson['variables'][variable]['label']
                variables = {k: v for k, v in sorted(variables.items(), key=lambda item: item[0])}
                groups[group]['variables'] = variables

In [ ]:
groups = {k: v for k, v in sorted(groups.items(), key=lambda item: item[0])}

In [ ]:
import json
# with open('../morpc/census/acs_variable_groups.json', 'w') as file:
#     json.dump(groups, file, indent=3)

In [10]:
varjson['variables']['B01001_001E']['group']

'B01001'